# Convert CLEO file to different formats

In [9]:
import cleo_tools as cleo
import numpy as np
import h5py
from math import sqrt

import zipfile

import matplotlib.pylab as plt
import seaborn as sn

%matplotlib notebook

In [4]:
infilename = 'data31_100k_LARGE.dat'
#infilename = 'data31_500k.dat'
#infilename = 'data31.dat'


infile = open(infilename)
collisions = cleo.get_collisions(infile)

In [5]:
# Parse the data so that we can have
# 
# Number of each type of particle.
# Store the information about the particles in 2D blocks. 
#     This works if all the data are the same file type (e.g. floats)

lp = []
lk = []
lm = []
le = []
lph = []

pis = []
kas = []
mus = []
els = []
phs = []

for collision in collisions:
    
    pions,kaons,muons,electrons,photons = collision
    
    npi = len(pions)
    nka = len(kaons)
    nmu = len(muons)
    nel = len(electrons)
    nph = len(photons)
    
    lp.append(npi)
    lk.append(nka)
    lm.append(nmu)
    le.append(nel)
    lph.append(nph)
    
    for pion in pions:
        
        pis.append(pion)
    
    for kaon in kaons:
        
        kas.append(kaon)
        
    for muon in muons:
        
        mus.append(muon)
    
    for electron in electrons:
        
        els.append(electron)
    
    for photon in photons:
        
        phs.append(photon)
    
    #print "event %i: |" % (i), "pions: %i | kaons: %i | muons: %i | electrons: %i | photons: %i | " % (npi,nka,nmu,nel,nph)


In [7]:
# Make the HDF file with different compressions

comp_tags = ['NO_COMPRESSION','GZIP','LZF']
algs = [None,'gzip','lzf']
compression_levels = [None,9,None]


for tag,alg,opt in zip(comp_tags,algs,compression_levels):
    
    end_tag = "_%s.hdf5" % (tag)
    outfilename = infilename.replace('.dat',end_tag)
    
    print outfilename

    with h5py.File(outfilename,'a') as hf:
        hf.create_dataset('npions',data=lp,compression=alg,compression_opts=opt)
        hf.create_dataset('nkaons',data=lk,compression=alg,compression_opts=opt)
        hf.create_dataset('nmuons',data=lm,compression=alg,compression_opts=opt)
        hf.create_dataset('nelectrons',data=le,compression=alg,compression_opts=opt)
        hf.create_dataset('nphotons',data=lph,compression=alg,compression_opts=opt)

        g1 = hf.create_group('particles')

        g1.create_dataset('pions',data=pis,compression=alg,compression_opts=opt)
        g1.create_dataset('kaons',data=kas,compression=alg,compression_opts=opt)
        g1.create_dataset('muons',data=mus,compression=alg,compression_opts=opt)
        g1.create_dataset('electrons',data=els,compression=alg,compression_opts=opt)
        g1.create_dataset('photons',data=phs,compression=alg,compression_opts=opt)

data31_100k_LARGE_NO_COMPRESSION.hdf5
data31_100k_LARGE_GZIP.hdf5
data31_100k_LARGE_LZF.hdf5


In [11]:
# Make a zipfile 

zipfilename = "%s.zip" % (infilename.split('.dat')[0])
zf = zipfile.ZipFile(zipfilename,'w')
zf.write(infilename.split(',dat')[0],compress_type=zipfile.ZIP_DEFLATED)
zf.close()
